In [3]:
from pprint import pprint

from module_.utils.tf_agent import TF_Agent

In [4]:
agent = TF_Agent()
agent.set_hyper_parameters(
    batch_size=256
)
agent.init_graph()
agent.init_sess()

In [ ]:
for _ in range(3):
    agent.run_sess()

 2.72% | acc-l 0.2656250 | acc-v 0.5000000 | acc-l-s 0.9960938 | acc-v-s 1.0000000 | loss-gen 0.8668308

In [9]:
pprint(agent.check_sen(777))

100.00%
('암만 오리지널 씬 이 후일담 형식 으로 있 긴 있 지만 그래도 티 비판 총 집편 이 다 재밌 을 리 가 있 나 난 총 집편 이 진짜 싫 '
 '드라 이렇게 때우 는 게 뭐 가 극장판',
 '금금 극한값 가로등 이 쭈우우우욱 필하모니 으로 있 이외 있 지만 글라스노스트 통단 교통경찰 씰 이 금금 재미있 을 덴데 가 있 다 '
 '전재일 교통경찰 씰 이 서각 메스꺼웠 괜츈한듯 꾸뻑 군더 는 게 만질수록 가 사진학',
 {'fake': {'label': (array([0, 1]),
                     array([[0.9828646 , 0.01713534]], dtype=float32),
                     0.0),
           'valid': (array([1, 0]),
                     array([[2.7313083e-06, 9.9999726e-01]], dtype=float32),
                     0.5)},
  'real': {'label': (array([1, 0]),
                     array([[0.00457131, 0.9954287 ]], dtype=float32),
                     0.5),
           'valid': (array([0, 1]),
                     array([[2.7311103e-06, 9.9999726e-01]], dtype=float32),
                     0.5)}})


In [ ]:
print('\r1\r2')